# Accuracy Assessment_Soil Moisture
- We compare the SMAP Soil Moisture data and NDAWN station Soil Moisture data

#### Package

In [1]:
import arcpy
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#### Workspace

In [2]:
arcpy.env.workspace = r"D:\spring2024\GIS5572\Final\Final project GIS5572.gdb"

### Accuracy Assessment

In [3]:
def differences_soil(true_points, compare_points):
    
    # get the name of interpolation method as part of the output point layer 
    comparation = os.path.basename(compare_points)
    
    # spatal join
    output_join = f"diff_{comparation}"
    arcpy.analysis.SpatialJoin(true_points, compare_points, output_join, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "CLOSEST")

    # rename column
    arcpy.management.AlterField(output_join, "vwc", "ground_truth", "ground_truth")
    arcpy.management.AlterField(output_join, "grid_code", "compare", "compare")

    # calculate difference
    fields = ["station_name", "ground_truth", "compare", "residual"] # vwc = Volumetric Water Conten
    arcpy.management.AddField(output_join, "residual", "DOUBLE")
    with arcpy.da.UpdateCursor(output_join, fields) as cursor:
        for row in cursor:
            row[3] = row[1] - row[2]  # "ground_truth" - "prediction"
            cursor.updateRow(row)
    arcpy.management.DeleteField(output_join,"Join_Count;TARGET_FID;elevation;pointid;year;month;day","DELETE_FIELDS")
    
    print(f"Difference {comparation} points layer created.")


In [4]:
true_points = "Soil_Clip"
compare_points = "Soil_Moisture_202404_pm_Clip"


differences_soil(true_points, compare_points)


Difference Soil_Moisture_202404_pm_Clip points layer created.


### Defince the RMSE, MAE, R^2 function

In [5]:
# RMSE
def RMSE(data):
    residual_squared_sum = 0
    count = 0
    for row in arcpy.da.SearchCursor(data, ['residual']):
        residual_squared_sum += row[0] ** 2
        count += 1
            
    rmse = (residual_squared_sum / count) ** 0.5
    return rmse



In [6]:
# Mean Absolute Error
def MAE(data):
    absolute_errors = []
    with arcpy.da.SearchCursor(data, ["ground_truth", "compare"]) as cursor:
        for row in cursor:
            ground_truth = row[0]
            prediction = row[1]
            absolute_errors.append(abs(ground_truth - prediction))
    
    
    mae = sum(absolute_errors) / len(absolute_errors)
    return mae


In [7]:
# Coefficient Of Determination
def R_squared(data):
    y_true = []
    y_pred = []
    
    with arcpy.da.SearchCursor(data, ["ground_truth", "compare"]) as cursor:
        for row in cursor:
            y_true.append(row[0])
            y_pred.append(row[1])
    
    y_mean = sum(y_true) / len(y_true)    
    ss_total = sum((y_i - y_mean) ** 2 for y_i in y_true)   
    ss_residual = sum((y_true[i] - y_pred[i]) ** 2 for i in range(len(y_true)))   
    r_squared = 1 - (ss_residual / ss_total)   
    return r_squared


In [9]:
diff_result = "diff_Soil_Moisture_202404_pm_Clip"

print("Soil_Moisture RMSE:", RMSE(diff_result))
    
print("Soil_Moisture MAE:", MAE(diff_result))

print("Soil_Moisture R^2:", R_squared(diff_result))

Soil_Moisture RMSE: 0.136732249262533
Soil_Moisture MAE: 0.12204098290205002
Soil_Moisture R^2: -2.0927272407297624
